In [1]:
!pip install simpletransformers
!pip install torch scikit-learn

import pandas as pd
import numpy as np
import torch
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import re


device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device_type}")


def simple_preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) 
    return text.lower()

# Load the data
file_path = '/kaggle/input/hellofile/combined_file.csv'
raw_data = pd.read_csv(file_path)
raw_data = raw_data.head(15000)
raw_data = raw_data.drop(['Unnamed: 5', 'Unnamed: 6'], axis=1)


processed_data = raw_data[['text', 'score']].dropna()
processed_data = processed_data[pd.to_numeric(processed_data['score'], errors='coerce').notna()]
processed_data['score'] = processed_data['score'].astype(float)
processed_data['text'] = processed_data['text'].apply(simple_preprocess)


k_folds = 10
mse_results = []


kf = KFold(n_splits=k_folds, shuffle=True, random_state=123)
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(processed_data), 1):
    train_data = processed_data.iloc[train_idx].rename(columns={'score': 'labels'})
    val_data = processed_data.iloc[val_idx].rename(columns={'score': 'labels'})
    

    xlnet_regressor = ClassificationModel(
        model_type="xlnet",
        model_name="xlnet-base-cased",
        num_labels=1,
        args={
            "num_train_epochs": 2,
            "train_batch_size": 16,
            "eval_batch_size": 16,
            "regression": True,
            "output_dir": f"/kaggle/working/xlnet_output_fold_{fold_idx}/",  
            "fp16": True,
            "use_multiprocessing": False,
            "overwrite_output_dir": True,  
        },
        use_cuda=(device_type == 'cuda')
    )


    xlnet_regressor.train_model(train_data)
    _, predictions, _ = xlnet_regressor.eval_model(val_data)


    actual_scores = val_data['labels'].values
    mse = mean_squared_error(actual_scores, predictions.flatten())
    mse_results.append(mse)



average_mse = np.mean(mse_results)
lowest_mse = np.min(mse_results)

print(f"Average MSE across all folds: {average_mse:.4f}")
print(f"Lowest MSE observed: {lowest_mse:.4f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 90.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=4cdc85eaf50140b6d6a7a1ef896d687f577c82b04c2fc6dafaabfa57ccf6b5d3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Using device: cuda


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	-

0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Average MSE across all folds: 94.5376
Lowest MSE observed: 0.0069
